# The Investigator's Report: Outbreak Investigation Dashboard

This notebook is the definitive tool for reviewing the complete findings of a field investigation. It serves as the primary interface for epidemiologists, program managers, and other stakeholders to access and understand the rich, detailed data collected during an investigation.

**Key Features:**
1.  **Investigation Selector:** Allows a user to select any completed investigation from the database.
2.  **Multi-Tabbed Report:** Presents the data in a comprehensive, organized format:
    *   **At a Glance:** Core details, clinical presentation, and population characteristics.
    *   **Risk Pathway Analysis:** A visual summary and detailed breakdown of the investigator's conclusions on likely outbreak sources.
    *   **Diagnostics:** A clear table of all laboratory results.
    *   **Surroundings:** A table detailing other relevant animal premises in the vicinity.

**Architecture:** This dashboard is powered by a highly efficient PostgreSQL function (`get_full_investigation_report`) that fetches all required data for a single investigation in one call, ensuring a fast and responsive user experience.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import json
import matplotlib.pyplot as plt
import io
import base64
from supabase import create_client, Client
from dotenv import load_dotenv
import os

# Load environment variables and connect to Supabase
load_dotenv()
supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_ANON_KEY")
if supabase_url and supabase_key:
    supabase: Client = create_client(supabase_url, supabase_key)
    print("✅ Successfully connected to Supabase.")
else:
    print("❌ Supabase credentials not found.")

### Part 1: Helper Functions & Data Loading

First, we define a helper function to embed Matplotlib charts in our dashboard. Then, we fetch a list of all available investigations from the database to populate our main selector widget. This ensures the user can choose from any report in the system.

In [ ]:
def plt_to_base64_html(plt_figure):
    buf = io.BytesIO()
    plt_figure.savefig(buf, format='png', bbox_inches='tight')
    img_b64 = base64.b64encode(buf.getvalue()).decode('utf-8')
    plt.close(plt_figure)
    return f'<img src="data:image/png;base64,{img_b64}" style="max-width:100%; height:auto;">'

# Fetch the list of all completed investigations to populate the dropdown
df_investigations = pd.DataFrame()
if 'supabase' in globals():
    try:
        res = supabase.table("outbreak_investigations").select("investigation_id, site_name, primary_species_investigated").execute()
        df_investigations = pd.DataFrame(res.data)
        print(f"✅ Found {len(df_investigations)} investigations to review.")
    except Exception as e:
        print(f"❌ Could not fetch list of investigations: {e}")

### Part 2: The Interactive Investigation Dashboard

This is the core of the notebook. It defines the widgets, the multi-tabbed layout, and the `update_dashboard` function. When a user selects an investigation, this function calls the `get_full_investigation_report` database function to retrieve a complete JSON object of the report and then populates all the tabs with the relevant information.

In [ ]:
if not df_investigations.empty:
    # --- 1. DEFINE WIDGETS AND LAYOUT ---
    investigation_options = {f"ID {r.investigation_id}: {r.site_name} ({r.primary_species_investigated})": r.investigation_id for _, r in df_investigations.iterrows()}
    investigation_selector = widgets.Dropdown(options={**{'-- Select an Investigation --': 0}, **investigation_options}, description='Investigation:')
    
    tab_summary = widgets.HTML()
    tab_risk = widgets.VBox()
    tab_diagnostics = widgets.HTML()
    tab_surroundings = widgets.HTML()
    
    main_tabs = widgets.Tab(children=[tab_summary, tab_risk, tab_diagnostics, tab_surroundings])
    main_tabs.set_title(0, 'At a Glance'); main_tabs.set_title(1, 'Risk Pathway Analysis'); main_tabs.set_title(2, 'Diagnostics'); main_tabs.set_title(3, 'Surroundings')

    # --- 2. THE MAIN UPDATE FUNCTION ---
    def update_dashboard(change):
        selected_id = investigation_selector.value
        if selected_id == 0:
            tab_summary.value = tab_diagnostics.value = tab_surroundings.value = "<p><i>Please select an investigation to view details.</i></p>"
            tab_risk.children = []
            return
            
        response = supabase.rpc('get_full_investigation_report', {'p_investigation_id': selected_id}).execute()
        if not response.data or not response.data[0]: return
        data = response.data[0]

        tab_summary.value = f"<h3>Site: {data.get('site_name', 'N/A')}</h3><p><b>Primary Species:</b> {data.get('primary_species', 'N/A')}</p><h4>Clinical Presentation</h4><p>{data.get('clinical_presentation', 'N/A')}</p>"

        risk_assessments = data.get('risk_assessments') or []
        if risk_assessments:
            df_risk = pd.DataFrame(risk_assessments)
            risk_counts = df_risk['likelihood_of_introduction'].value_counts()
            fig, ax = plt.subplots(figsize=(8,4)); risk_counts.plot(kind='bar', ax=ax, color=['red','orange','green','grey'])
            ax.set_title('Summary of Assessed Pathway Risks'); chart_html = plt_to_base64_html(fig)
            
            accordion_items = [widgets.HTML(f"<p><b>Justification:</b> {item.get('justification_for_risk', '')}</p><pre>{json.dumps(item.get('pathway_details', {}), indent=2)}</pre>") for item in risk_assessments]
            accordion = widgets.Accordion(children=accordion_items)
            for i, item in enumerate(risk_assessments): accordion.set_title(i, f"{item['pathway_category']} (Risk: {item['likelihood_of_introduction']})")
            tab_risk.children = [widgets.HTML(chart_html), accordion]
        else:
            tab_risk.children = [widgets.HTML("<p><i>No risk pathway assessments recorded.</i></p>")]

        tab_diagnostics.value = pd.DataFrame(data.get('diagnostic_tests', [])).to_html(index=False, classes='table') or "<p><i>No diagnostic tests recorded.</i></p>"
        tab_surroundings.value = pd.DataFrame(data.get('nearby_premises', [])).to_html(index=False, classes='table') or "<p><i>No nearby premises recorded.</i></p>"

    # --- 3. ASSEMBLE AND DISPLAY ---
    investigation_selector.observe(update_dashboard, names='value')
    dashboard = widgets.VBox([
        widgets.HTML("<h2>Outbreak Investigation Report Explorer</h2>"),
        investigation_selector, widgets.HTML("<hr>"), main_tabs
    ])
    display(dashboard)
    update_dashboard(None)
else:
    print("⚠️ Dashboard cannot be displayed as no investigation data was loaded.")